In [30]:
# Dependencies
import gmaps
import numpy as np
import pandas as pd
import requests
import sys
import os
from matplotlib import pyplot as plt


# Configure gmaps
gmaps.configure(api_key=gkey)

In [31]:
#Load csv in previews part
city_data_df = pd.read_csv( "../WeatherPy/cities.csv")
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.60,-73.97,80.276,53,0,2.00,US,1596209588
1,hofn,64.25,-15.21,53.870,87,90,7.70,IS,1596209707
2,ranau,5.95,116.66,79.070,83,20,2.10,MY,1596209708
3,sao filipe,14.90,-24.50,76.100,77,41,4.80,CV,1596209708
4,olafsvik,64.89,-23.71,51.800,88,100,8.95,IS,1596209709


In [32]:
# h
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

In [33]:
#Create new DataFrame fitting weather criteria
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,hami,42.80,93.45,79.196,17,0,1.44,CN,1596209720
111,ponta do sol,32.67,-17.10,75.272,62,0,0.45,PT,1596209748
132,yining,29.04,114.56,78.638,87,0,0.94,CN,1596209756
174,zonguldak,41.25,31.83,75.092,64,0,2.40,TR,1596209779
178,skjervoy,70.03,20.97,70.070,56,0,6.70,NO,1596209781
198,copiapo,-27.37,-70.33,71.870,33,0,5.10,CL,1596209790
231,hwange,-18.37,26.50,72.788,21,0,1.76,ZW,1596209810
248,calama,-22.47,-68.93,71.870,6,0,4.10,CL,1596209565
356,saldanha,-33.01,17.94,70.070,56,0,4.10,ZA,1596209868
435,namibe,-15.20,12.15,71.870,73,0,5.10,AO,1596209896


# Hotel Map

In [34]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
33,hami,CN,42.80,93.45,
111,ponta do sol,PT,32.67,-17.10,
132,yining,CN,29.04,114.56,
174,zonguldak,TR,41.25,31.83,
178,skjervoy,NO,70.03,20.97,
198,copiapo,CL,-27.37,-70.33,
231,hwange,ZW,-18.37,26.50,
248,calama,CL,-22.47,-68.93,
356,saldanha,ZA,-33.01,17.94,
435,namibe,AO,-15.20,12.15,


In [36]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = str(lat)+","+str(lng)
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Lat,Lng,Hotel Name
33,hami,CN,42.80,93.45,Laoyutang Resort
111,ponta do sol,PT,32.67,-17.10,Hotel do Campo
132,yining,CN,29.04,114.56,Xiushui Hotel
174,zonguldak,TR,41.25,31.83,Süleyman Tez Ev
178,skjervoy,NO,70.03,20.97,Skjervøy Byggeservice AS
198,copiapo,CL,-27.37,-70.33,Antay Casino Hotel
231,hwange,ZW,-18.37,26.50,Hwange Colliery Guest House
248,calama,CL,-22.47,-68.93,Park Hotel Calama
356,saldanha,ZA,-33.01,17.94,Blue Bay Lodge
435,namibe,AO,-15.20,12.15,Hotel Chik Chik Namibe


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))